In [1]:
#Import Required Packages
import pandas as pd
import requests
import json
import io


import warnings
warnings.filterwarnings('ignore')

import urllib3
from urllib3 import request

# Part 1: HDB Resale (DataFrame 1)

In [2]:
import urllib3
from urllib3 import request

#Extract HDB Resale Data from Gov Data API
http = urllib3.PoolManager()

url = 'https://data.gov.sg/api/action/datastore_search?resource_id=f1765b54-a209-4718-8d38-a39237f502b3&limit=100000'
req = http.request('GET', url)
req.status

200

In [3]:
'https://data.gov.sg/api/action/datastore_search?resource_id=f1765b54-a209-4718-8d38-a39237f502b3&limit=5&q=title:jones'

'https://data.gov.sg/api/action/datastore_search?resource_id=f1765b54-a209-4718-8d38-a39237f502b3&limit=5&q=title:jones'

In [3]:
#Extract Data in JSON Format
data = json.loads(req.data)
results = data['result']
results

{'resource_id': 'f1765b54-a209-4718-8d38-a39237f502b3',
 'fields': [{'type': 'int4', 'id': '_id'},
  {'type': 'text', 'id': 'month'},
  {'type': 'text', 'id': 'town'},
  {'type': 'text', 'id': 'flat_type'},
  {'type': 'text', 'id': 'block'},
  {'type': 'text', 'id': 'street_name'},
  {'type': 'text', 'id': 'storey_range'},
  {'type': 'numeric', 'id': 'floor_area_sqm'},
  {'type': 'text', 'id': 'flat_model'},
  {'type': 'numeric', 'id': 'lease_commence_date'},
  {'type': 'text', 'id': 'remaining_lease'},
  {'type': 'numeric', 'id': 'resale_price'}],
 'records': [{'town': 'WOODLANDS',
   'flat_type': '5 ROOM',
   'flat_model': 'Model A',
   'floor_area_sqm': '134',
   'street_name': 'MARSILING RD',
   'resale_price': '350000',
   'month': '2017-03',
   'remaining_lease': '77 years 01 month',
   'lease_commence_date': '1995',
   'storey_range': '04 TO 06',
   '_id': 1,
   'block': '139'},
  {'town': 'WOODLANDS',
   'flat_type': '5 ROOM',
   'flat_model': 'Standard',
   'floor_area_sqm': '

### Convert Json to Data Frame

In [31]:
df = pd.json_normalize(results, 'records')
df2 = df[49678:70000]
df2

,town,flat_type,flat_model,floor_area_sqm,street_name,resale_price,month,remaining_lease,lease_commence_date,storey_range,_id,block
49678,PUNGGOL,5 ROOM,Improved,110,EDGEDALE PLAINS,428000,2019-07,82 years 07 months,2003,10 TO 12,49678,126B
49679,PUNGGOL,5 ROOM,Improved,110,PUNGGOL FIELD,430000,2019-07,82 years 10 months,2003,13 TO 15,49679,196C
49680,PUNGGOL,5 ROOM,Premium Apartment,110,PUNGGOL CTRL,432000,2019-07,84 years 06 months,2005,13 TO 15,49680,645
49681,PUNGGOL,5 ROOM,Improved,110,EDGEFIELD PLAINS,445000,2019-07,82 years 09 months,2003,10 TO 12,49681,107C
49682,PUNGGOL,5 ROOM,Improved,110,EDGEFIELD PLAINS,445000,2019-07,82 years 10 months,2003,10 TO 12,49682,103A
...,...,...,...,...,...,...,...,...,...,...,...,...
69995,JURONG EAST,5 ROOM,Model A,132,JURONG EAST ST 32,465000,2020-07,63 years 01 month,1984,07 TO 09,69996,316
69996,JURONG EAST,5 ROOM,Standard,129,PANDAN GDNS,360000,2020-07,57 years 09 months,1979,01 TO 03,69997,407
69997,JURONG EAST,5 ROOM,Standard,104,PANDAN GDNS,310000,2020-07,57 years 09 months,1979,04 TO 06,69998,404
69998,JURONG EAST,5 ROOM,Improved,115,TEBAN GDNS RD,415000,2020-07,81 years 11 months,2003,13 TO 15,69999,63


# Part 2: One MAP API (DataFrame 2)

### Create Address List

In [32]:
df2['address'] = df2['block'] + " " + df2['street_name']
addresslist = list(df2['address'])
addresslist

['126B EDGEDALE PLAINS',
 '196C PUNGGOL FIELD',
 '645 PUNGGOL CTRL',
 '107C EDGEFIELD PLAINS',
 '103A EDGEFIELD PLAINS',
 '171C EDGEDALE PLAINS',
 '652 PUNGGOL CTRL',
 '116 EDGEFIELD PLAINS',
 '649 PUNGGOL CTRL',
 '293 PUNGGOL CTRL',
 '296 PUNGGOL CTRL',
 '161B PUNGGOL CTRL',
 '170A PUNGGOL FIELD',
 '270C PUNGGOL FIELD',
 '213B PUNGGOL WALK',
 '213B PUNGGOL WALK',
 '207B PUNGGOL PL',
 '264A PUNGGOL WAY',
 '207D PUNGGOL PL',
 '618B PUNGGOL DR',
 '266A PUNGGOL WAY',
 '271C PUNGGOL WALK',
 '288A PUNGGOL PL',
 '440A FERNVALE LINK',
 '289C PUNGGOL PL',
 '271C PUNGGOL WALK',
 '267B PUNGGOL FIELD',
 "83 C'WEALTH CL",
 "84 C'WEALTH CL",
 "101 C'WEALTH CRES",
 '18 GHIM MOH RD',
 '169 STIRLING RD',
 "92 C'WEALTH DR",
 '171 STIRLING RD',
 '152 MEI LING ST',
 "99 C'WEALTH CRES",
 '170 STIRLING RD',
 "98 C'WEALTH CRES",
 '165 STIRLING RD',
 "112 C'WEALTH CRES",
 "81 C'WEALTH CL",
 "113 C'WEALTH CRES",
 '152 MEI LING ST',
 '160 MEI LING ST',
 '2 GHIM MOH RD',
 '40 HOLLAND DR',
 '6 GHIM MOH RD',
 '4 

In [17]:
#Method to Extract Information of Address
def getcoordinates(address):
    request = requests.get('https://developers.onemap.sg/commonapi/search?searchVal='+ str(address) 
                           +'&returnGeom=Y&getAddrDetails=Y', verify=False)
    
    resultsdict = eval(request.text)
    
    if len(resultsdict['results'])>0:
        return resultsdict['results'][0]['POSTAL']
    else:
        pass

In [18]:
#Extraction of postal code from OneMap API 
postal_list3= []
count = 0
failed_count = 0
for address in addresslist:
    try:
        if len(getcoordinates(address))>0:
            count = count + 1
            print('Extracting',count,'out of',len(addresslist),'addresses')
            postal_list3.append(getcoordinates(address))
    except:
        count = count + 1           
        failed_count = failed_count + 1
        print('Failed to extract',count,'out of',len(addresslist),'addresses')
        postal_list3.append(None)
print('Total Number of Addresses With No Coordinates',failed_count)

Extracting 1 out of 29999 addresses
Extracting 2 out of 29999 addresses
Extracting 3 out of 29999 addresses
Extracting 4 out of 29999 addresses
Extracting 5 out of 29999 addresses
Extracting 6 out of 29999 addresses
Extracting 7 out of 29999 addresses
Extracting 8 out of 29999 addresses
Extracting 9 out of 29999 addresses
Extracting 10 out of 29999 addresses
Extracting 11 out of 29999 addresses
Extracting 12 out of 29999 addresses
Extracting 13 out of 29999 addresses
Extracting 14 out of 29999 addresses
Extracting 15 out of 29999 addresses
Extracting 16 out of 29999 addresses
Extracting 17 out of 29999 addresses
Extracting 18 out of 29999 addresses
Extracting 19 out of 29999 addresses
Extracting 20 out of 29999 addresses
Extracting 21 out of 29999 addresses
Extracting 22 out of 29999 addresses
Extracting 23 out of 29999 addresses
Extracting 24 out of 29999 addresses
Extracting 25 out of 29999 addresses
Extracting 26 out of 29999 addresses
Extracting 27 out of 29999 addresses
Extracting

Extracting 219 out of 29999 addresses
Extracting 220 out of 29999 addresses
Extracting 221 out of 29999 addresses
Extracting 222 out of 29999 addresses
Extracting 223 out of 29999 addresses
Extracting 224 out of 29999 addresses
Extracting 225 out of 29999 addresses
Extracting 226 out of 29999 addresses
Extracting 227 out of 29999 addresses
Extracting 228 out of 29999 addresses
Extracting 229 out of 29999 addresses
Extracting 230 out of 29999 addresses
Extracting 231 out of 29999 addresses
Extracting 232 out of 29999 addresses
Extracting 233 out of 29999 addresses
Extracting 234 out of 29999 addresses
Extracting 235 out of 29999 addresses
Extracting 236 out of 29999 addresses
Extracting 237 out of 29999 addresses
Extracting 238 out of 29999 addresses
Extracting 239 out of 29999 addresses
Extracting 240 out of 29999 addresses
Extracting 241 out of 29999 addresses
Extracting 242 out of 29999 addresses
Extracting 243 out of 29999 addresses
Extracting 244 out of 29999 addresses
Extracting 2

Extracting 435 out of 29999 addresses
Extracting 436 out of 29999 addresses
Extracting 437 out of 29999 addresses
Extracting 438 out of 29999 addresses
Extracting 439 out of 29999 addresses
Extracting 440 out of 29999 addresses
Extracting 441 out of 29999 addresses
Extracting 442 out of 29999 addresses
Extracting 443 out of 29999 addresses
Extracting 444 out of 29999 addresses
Extracting 445 out of 29999 addresses
Extracting 446 out of 29999 addresses
Extracting 447 out of 29999 addresses
Extracting 448 out of 29999 addresses
Extracting 449 out of 29999 addresses
Extracting 450 out of 29999 addresses
Extracting 451 out of 29999 addresses
Extracting 452 out of 29999 addresses
Extracting 453 out of 29999 addresses
Extracting 454 out of 29999 addresses
Extracting 455 out of 29999 addresses
Extracting 456 out of 29999 addresses
Extracting 457 out of 29999 addresses
Extracting 458 out of 29999 addresses
Extracting 459 out of 29999 addresses
Extracting 460 out of 29999 addresses
Extracting 4

Extracting 651 out of 29999 addresses
Extracting 652 out of 29999 addresses
Extracting 653 out of 29999 addresses
Extracting 654 out of 29999 addresses
Extracting 655 out of 29999 addresses
Extracting 656 out of 29999 addresses
Extracting 657 out of 29999 addresses
Extracting 658 out of 29999 addresses
Extracting 659 out of 29999 addresses
Extracting 660 out of 29999 addresses
Extracting 661 out of 29999 addresses
Extracting 662 out of 29999 addresses
Extracting 663 out of 29999 addresses
Extracting 664 out of 29999 addresses
Extracting 665 out of 29999 addresses
Extracting 666 out of 29999 addresses
Extracting 667 out of 29999 addresses
Extracting 668 out of 29999 addresses
Extracting 669 out of 29999 addresses
Extracting 670 out of 29999 addresses
Extracting 671 out of 29999 addresses
Extracting 672 out of 29999 addresses
Extracting 673 out of 29999 addresses
Extracting 674 out of 29999 addresses
Extracting 675 out of 29999 addresses
Extracting 676 out of 29999 addresses
Extracting 6

Extracting 867 out of 29999 addresses
Extracting 868 out of 29999 addresses
Extracting 869 out of 29999 addresses
Extracting 870 out of 29999 addresses
Extracting 871 out of 29999 addresses
Extracting 872 out of 29999 addresses
Extracting 873 out of 29999 addresses
Extracting 874 out of 29999 addresses
Extracting 875 out of 29999 addresses
Extracting 876 out of 29999 addresses
Extracting 877 out of 29999 addresses
Extracting 878 out of 29999 addresses
Extracting 879 out of 29999 addresses
Extracting 880 out of 29999 addresses
Extracting 881 out of 29999 addresses
Extracting 882 out of 29999 addresses
Extracting 883 out of 29999 addresses
Extracting 884 out of 29999 addresses
Extracting 885 out of 29999 addresses
Extracting 886 out of 29999 addresses
Extracting 887 out of 29999 addresses
Extracting 888 out of 29999 addresses
Extracting 889 out of 29999 addresses
Extracting 890 out of 29999 addresses
Extracting 891 out of 29999 addresses
Extracting 892 out of 29999 addresses
Extracting 8

...

Extracting 29000 out of 29999 addresses
Extracting 29001 out of 29999 addresses
Extracting 29002 out of 29999 addresses
Extracting 29003 out of 29999 addresses
Extracting 29004 out of 29999 addresses
Extracting 29005 out of 29999 addresses
Extracting 29006 out of 29999 addresses
Extracting 29007 out of 29999 addresses
Extracting 29008 out of 29999 addresses
Extracting 29009 out of 29999 addresses
Extracting 29010 out of 29999 addresses
Extracting 29011 out of 29999 addresses
Extracting 29012 out of 29999 addresses
Extracting 29013 out of 29999 addresses
Extracting 29014 out of 29999 addresses
Extracting 29015 out of 29999 addresses
Extracting 29016 out of 29999 addresses
Extracting 29017 out of 29999 addresses
Extracting 29018 out of 29999 addresses
Extracting 29019 out of 29999 addresses
Extracting 29020 out of 29999 addresses
Extracting 29021 out of 29999 addresses
Extracting 29022 out of 29999 addresses
Extracting 29023 out of 29999 addresses
Extracting 29024 out of 29999 addresses


Extracting 29205 out of 29999 addresses
Extracting 29206 out of 29999 addresses
Extracting 29207 out of 29999 addresses
Extracting 29208 out of 29999 addresses
Extracting 29209 out of 29999 addresses
Extracting 29210 out of 29999 addresses
Extracting 29211 out of 29999 addresses
Extracting 29212 out of 29999 addresses
Extracting 29213 out of 29999 addresses
Extracting 29214 out of 29999 addresses
Extracting 29215 out of 29999 addresses
Extracting 29216 out of 29999 addresses
Extracting 29217 out of 29999 addresses
Extracting 29218 out of 29999 addresses
Extracting 29219 out of 29999 addresses
Extracting 29220 out of 29999 addresses
Extracting 29221 out of 29999 addresses
Extracting 29222 out of 29999 addresses
Extracting 29223 out of 29999 addresses
Extracting 29224 out of 29999 addresses
Extracting 29225 out of 29999 addresses
Extracting 29226 out of 29999 addresses
Extracting 29227 out of 29999 addresses
Extracting 29228 out of 29999 addresses
Extracting 29229 out of 29999 addresses


Extracting 29410 out of 29999 addresses
Extracting 29411 out of 29999 addresses
Extracting 29412 out of 29999 addresses
Extracting 29413 out of 29999 addresses
Extracting 29414 out of 29999 addresses
Extracting 29415 out of 29999 addresses
Extracting 29416 out of 29999 addresses
Extracting 29417 out of 29999 addresses
Extracting 29418 out of 29999 addresses
Extracting 29419 out of 29999 addresses
Extracting 29420 out of 29999 addresses
Extracting 29421 out of 29999 addresses
Extracting 29422 out of 29999 addresses
Extracting 29423 out of 29999 addresses
Extracting 29424 out of 29999 addresses
Extracting 29425 out of 29999 addresses
Extracting 29426 out of 29999 addresses
Extracting 29427 out of 29999 addresses
Extracting 29428 out of 29999 addresses
Extracting 29429 out of 29999 addresses
Extracting 29430 out of 29999 addresses
Extracting 29431 out of 29999 addresses
Extracting 29432 out of 29999 addresses
Extracting 29433 out of 29999 addresses
Extracting 29434 out of 29999 addresses


Extracting 29615 out of 29999 addresses
Extracting 29616 out of 29999 addresses
Extracting 29617 out of 29999 addresses
Extracting 29618 out of 29999 addresses
Extracting 29619 out of 29999 addresses
Extracting 29620 out of 29999 addresses
Extracting 29621 out of 29999 addresses
Extracting 29622 out of 29999 addresses
Extracting 29623 out of 29999 addresses
Extracting 29624 out of 29999 addresses
Extracting 29625 out of 29999 addresses
Extracting 29626 out of 29999 addresses
Extracting 29627 out of 29999 addresses
Extracting 29628 out of 29999 addresses
Extracting 29629 out of 29999 addresses
Extracting 29630 out of 29999 addresses
Extracting 29631 out of 29999 addresses
Extracting 29632 out of 29999 addresses
Extracting 29633 out of 29999 addresses
Extracting 29634 out of 29999 addresses
Extracting 29635 out of 29999 addresses
Extracting 29636 out of 29999 addresses
Extracting 29637 out of 29999 addresses
Extracting 29638 out of 29999 addresses
Extracting 29639 out of 29999 addresses


Extracting 29820 out of 29999 addresses
Extracting 29821 out of 29999 addresses
Extracting 29822 out of 29999 addresses
Extracting 29823 out of 29999 addresses
Extracting 29824 out of 29999 addresses
Extracting 29825 out of 29999 addresses
Extracting 29826 out of 29999 addresses
Extracting 29827 out of 29999 addresses
Extracting 29828 out of 29999 addresses
Extracting 29829 out of 29999 addresses
Extracting 29830 out of 29999 addresses
Extracting 29831 out of 29999 addresses
Extracting 29832 out of 29999 addresses
Extracting 29833 out of 29999 addresses
Extracting 29834 out of 29999 addresses
Extracting 29835 out of 29999 addresses
Extracting 29836 out of 29999 addresses
Extracting 29837 out of 29999 addresses
Extracting 29838 out of 29999 addresses
Extracting 29839 out of 29999 addresses
Extracting 29840 out of 29999 addresses
Extracting 29841 out of 29999 addresses
Extracting 29842 out of 29999 addresses
Extracting 29843 out of 29999 addresses
Extracting 29844 out of 29999 addresses


In [33]:
postal_list2

[822126,
 823196,
 820645,
 823107,
 821103,
 823171,
 820652,
 820116,
 820649,
 820293,
 820296,
 822161,
 821170,
 823270,
 822213,
 822213,
 822207,
 821264,
 824207,
 822618,
 821266,
 823271,
 821288,
 791440,
 823289,
 823271,
 822267,
 140083,
 140084,
 140101,
 270018,
 140169,
 140092,
 140171,
 140152,
 140099,
 140170,
 140098,
 140165,
 140112,
 140081,
 140113,
 140152,
 140160,
 270002,
 270040,
 270006,
 271004,
 272009,
 272012,
 272012,
 141051,
 144061,
 270026,
 142050,
 270026,
 140168,
 271008,
 271006,
 271002,
 142091,
 132028,
 134028,
 140051,
 134028,
 144061,
 132028,
 142059,
 143062,
 141090,
 143061,
 270027,
 132028,
 142091,
 144061,
 271022,
 271023,
 142053,
 142053,
 274018,
 270007,
 270014,
 271021,
 130012,
 130013,
 140051,
 140184,
 141090,
 140148,
 140148,
 752593,
 751590,
 752593,
 753590,
 751590,
 752593,
 751591,
 752463,
 751591,
 752590,
 750314,
 750404,
 750474,
 750406,
 750486,
 750482,
 750411,
 751466,
 750486,
 750482,
 752589,
 

### Convert Postal Code from String to Integer

In [8]:
for i in range(0, len(postal_list)):
    postal_list3[i] = int(postal_list[i])

postal_list

TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'

In [34]:
#Add Postal Code into Data Frame
df['Postal'] = postal_list
df

,town,flat_type,flat_model,floor_area_sqm,street_name,resale_price,month,remaining_lease,lease_commence_date,storey_range,_id,block,address,Postal
49678,PUNGGOL,5 ROOM,Improved,110,EDGEDALE PLAINS,428000,2019-07,82 years 07 months,2003,10 TO 12,49678,126B,126B EDGEDALE PLAINS,822126
49679,PUNGGOL,5 ROOM,Improved,110,PUNGGOL FIELD,430000,2019-07,82 years 10 months,2003,13 TO 15,49679,196C,196C PUNGGOL FIELD,823196
49680,PUNGGOL,5 ROOM,Premium Apartment,110,PUNGGOL CTRL,432000,2019-07,84 years 06 months,2005,13 TO 15,49680,645,645 PUNGGOL CTRL,820645
49681,PUNGGOL,5 ROOM,Improved,110,EDGEFIELD PLAINS,445000,2019-07,82 years 09 months,2003,10 TO 12,49681,107C,107C EDGEFIELD PLAINS,823107
49682,PUNGGOL,5 ROOM,Improved,110,EDGEFIELD PLAINS,445000,2019-07,82 years 10 months,2003,10 TO 12,49682,103A,103A EDGEFIELD PLAINS,821103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,JURONG EAST,5 ROOM,Model A,132,JURONG EAST ST 32,465000,2020-07,63 years 01 month,1984,07 TO 09,69996,316,316 JURONG EAST ST 32,600316
69996,JURONG EAST,5 ROOM,Standard,129,PANDAN GDNS,360000,2020-07,57 years 09 months,1979,01 TO 03,69997,407,407 PANDAN GDNS,600407
69997,JURONG EAST,5 ROOM,Standard,104,PANDAN GDNS,310000,2020-07,57 years 09 months,1979,04 TO 06,69998,404,404 PANDAN GDNS,600404
69998,JURONG EAST,5 ROOM,Improved,115,TEBAN GDNS RD,415000,2020-07,81 years 11 months,2003,13 TO 15,69999,63,63 TEBAN GDNS RD,600063


In [20]:
#Add Postal Code into Data Frame
df3['Postal'] = postal_list3
df3

,town,flat_type,flat_model,floor_area_sqm,street_name,resale_price,month,remaining_lease,lease_commence_date,storey_range,_id,block,address,Postal
70001,JURONG EAST,5 ROOM,Improved,121,TOH GUAN RD,505000,2020-07,77 years 08 months,1999,01 TO 03,70002,270,270 TOH GUAN RD,600270
70002,JURONG EAST,EXECUTIVE,Maisonette,149,JURONG EAST AVE 1,510000,2020-07,64 years 01 month,1985,01 TO 03,70003,337,337 JURONG EAST AVE 1,600337
70003,JURONG EAST,EXECUTIVE,Maisonette,156,JURONG EAST AVE 1,545000,2020-07,64 years 01 month,1985,01 TO 03,70004,336,336 JURONG EAST AVE 1,600336
70004,JURONG EAST,EXECUTIVE,Apartment,144,TOH GUAN RD,755000,2020-07,77 years 06 months,1998,13 TO 15,70005,285A,285A TOH GUAN RD,601285
70005,JURONG EAST,EXECUTIVE,Apartment,144,TOH GUAN RD,770000,2020-07,77 years 05 months,1998,07 TO 09,70006,285D,285D TOH GUAN RD,604285
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,ANG MO KIO,3 ROOM,New Generation,68,ANG MO KIO AVE 5,315000,2021-08,58 years 09 months,1981,04 TO 06,99995,180,180 ANG MO KIO AVE 5,560180
99996,ANG MO KIO,3 ROOM,New Generation,67,ANG MO KIO AVE 5,280000,2021-08,58 years,1980,01 TO 03,99996,602,602 ANG MO KIO AVE 5,560602
99997,ANG MO KIO,3 ROOM,New Generation,68,ANG MO KIO AVE 6,348888,2021-08,58 years 02 months,1980,07 TO 09,99997,639,639 ANG MO KIO AVE 6,560639
99998,ANG MO KIO,3 ROOM,New Generation,67,ANG MO KIO AVE 6,355000,2021-08,62 years 06 months,1985,10 TO 12,99998,633,633 ANG MO KIO AVE 6,560633


In [35]:
df2.to_csv('postalcodes2.csv')

# Merge different csv files together

In [40]:
df_combined = pd.concat(
    map(pd.read_csv, ['postalcodes.csv', 'postalcodes2.csv', 'postalcodes3.csv']), ignore_index=True)
df_combined

,Unnamed: 0,town,flat_type,flat_model,floor_area_sqm,street_name,resale_price,month,remaining_lease,lease_commence_date,storey_range,_id,block,address,Postal
0,0,WOODLANDS,5 ROOM,Model A,134.0,MARSILING RD,350000.0,2017-03,77 years 01 month,1995,04 TO 06,1,139,139 MARSILING RD,730139
1,1,WOODLANDS,5 ROOM,Standard,120.0,MARSILING DR,355000.0,2017-03,57 years 10 months,1976,16 TO 18,2,9,9 MARSILING DR,730009
2,2,WOODLANDS,5 ROOM,Standard,126.0,MARSILING DR,370000.0,2017-03,58 years,1976,01 TO 03,3,10,10 MARSILING DR,730010
3,3,WOODLANDS,5 ROOM,Improved,120.0,WOODLANDS DR 60,370000.0,2017-03,79 years 07 months,1997,04 TO 06,4,786D,786D WOODLANDS DR 60,734786
4,4,WOODLANDS,5 ROOM,Premium Apartment,110.0,WOODLANDS DR 75,370000.0,2017-03,86 years,2004,10 TO 12,5,688B,688B WOODLANDS DR 75,732688
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99995,ANG MO KIO,3 ROOM,New Generation,68.0,ANG MO KIO AVE 5,315000.0,2021-08,58 years 09 months,1981,04 TO 06,99995,180,180 ANG MO KIO AVE 5,560180
99996,99996,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,280000.0,2021-08,58 years,1980,01 TO 03,99996,602,602 ANG MO KIO AVE 5,560602
99997,99997,ANG MO KIO,3 ROOM,New Generation,68.0,ANG MO KIO AVE 6,348888.0,2021-08,58 years 02 months,1980,07 TO 09,99997,639,639 ANG MO KIO AVE 6,560639
99998,99998,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 6,355000.0,2021-08,62 years 06 months,1985,10 TO 12,99998,633,633 ANG MO KIO AVE 6,560633


In [43]:
df_combined['Postal'].isnull().sum()


232

# Part 3: Identify HDB From Sample Data

In [122]:
#Load Sample Data
df2 = pd.read_csv("listings.csv")
df2

,postal,name,neighbourhood_group,neighbourhood,latitude,longitude,room_type
0,730139,Francesca,North Region,Woodlands,1.44255,103.79580,Private room
1,227796,Sujatha,Central Region,Bukit Timah,1.33235,103.78521,Private room
2,266763,Francesca,North Region,Woodlands,1.44246,103.79667,Private room
3,367042,Belinda,East Region,Tampines,1.34541,103.95712,Private room
4,367042,Belinda,East Region,Tampines,1.34567,103.95963,Private room
5,367042,Belinda,East Region,Tampines,1.34702,103.96103,Private room
6,367042,Belinda,East Region,Tampines,1.34348,103.96337,Private room
7,733688,Bianca,East Region,Bedok,1.32304,103.91363,Private room
8,786947,Bianca,East Region,Bedok,1.32458,103.91163,Private room
9,1017645,Bianca,East Region,Bedok,1.32461,103.91191,Private room


In [88]:
#df.to_json(r"C:\Users\sgpctf\OneDrive - FWD Group Management Holdings Limited/Desktop/HDB/customer.json")
#df

### Identify HDB from sample data

In [126]:
for i in df2['postal']:
    if(i in postal_list):
        print(i)

730139
733688
730808
760802
760746
731502
733787
760233


In [128]:
#List to store HDB value
HDB = []

#For loop to look through every row
for i in df2['postal']:
    if(i in postal_list):
        HDB.append(1)
    else:
        HDB.append(0)

In [130]:
df2['HDB'] = HDB
df2

,postal,name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,HDB
0,730139,Francesca,North Region,Woodlands,1.44255,103.79580,Private room,1
1,227796,Sujatha,Central Region,Bukit Timah,1.33235,103.78521,Private room,0
2,266763,Francesca,North Region,Woodlands,1.44246,103.79667,Private room,0
3,367042,Belinda,East Region,Tampines,1.34541,103.95712,Private room,0
4,367042,Belinda,East Region,Tampines,1.34567,103.95963,Private room,0
5,367042,Belinda,East Region,Tampines,1.34702,103.96103,Private room,0
6,367042,Belinda,East Region,Tampines,1.34348,103.96337,Private room,0
7,733688,Bianca,East Region,Bedok,1.32304,103.91363,Private room,1
8,786947,Bianca,East Region,Bedok,1.32458,103.91163,Private room,0
9,1017645,Bianca,East Region,Bedok,1.32461,103.91191,Private room,0


## Debugging Codes (not important)

In [35]:
x.keys()

dict_keys(['found', 'totalNumPages', 'pageNum', 'results'])

In [36]:
x['results']

[{'SEARCHVAL': '139 MARSILING ROAD SINGAPORE 730139',
  'BLK_NO': '139',
  'ROAD_NAME': 'MARSILING ROAD',
  'BUILDING': 'NIL',
  'ADDRESS': '139 MARSILING ROAD SINGAPORE 730139',
  'POSTAL': '730139',
  'X': '21763.2928561866',
  'Y': '46604.934607083',
  'LATITUDE': '1.43775226684572',
  'LONGITUDE': '103.777275768219',
  'LONGTITUDE': '103.777275768219'}]

In [38]:
df = pd.DataFrame(x['results'])
df

,SEARCHVAL,BLK_NO,ROAD_NAME,BUILDING,ADDRESS,POSTAL,X,Y,LATITUDE,LONGITUDE,LONGTITUDE
0,139 MARSILING ROAD SINGAPORE 730139,139,MARSILING ROAD,NIL,139 MARSILING ROAD SINGAPORE 730139,730139,21763.2928561866,46604.934607083,1.43775226684572,103.777275768219,103.777275768219
